In [1]:
import pickle
from albumentations import ShiftScaleRotate,Cutout,RandomContrast,RandomBrightness,Compose
from utility.albumentations_helper import create_transform
from utility.others import clear_all
import time
from functions import DataGenerator,PredictGenerator,create_model,dot_distance,exp_loss,\
                        sigmoid_dot_distance,dot_sigmoid_distance,cross_entropy_loss,l2_distance,\
                        margin_loss_fun_factory,Xception_reduced
from utility.grayscale_imagenet import Xception_greyscale
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model
#from tensorflow.keras.applications import DenseNet121

In [2]:
color = False

### set up generator

In [3]:
if color:
    with open('/home/will/Desktop/kaggle/Whale/train_df_color.pkl', 'rb') as f:
        Ids_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_train_color.pkl', 'rb') as f:
        newWhale_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/val_df_color.pkl', 'rb') as f:
        Ids_val = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_val_color.pkl', 'rb') as f:
        newWhale_val = pickle.load(f)
else:
    with open('/home/will/Desktop/kaggle/Whale/train_df.pkl', 'rb') as f:
        Ids_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_train.pkl', 'rb') as f:
        newWhale_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/val_df.pkl', 'rb') as f:
        Ids_val = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_val.pkl', 'rb') as f:
        newWhale_val = pickle.load(f)

In [4]:
aug = Compose([RandomContrast(p=0.2),RandomBrightness(p=0.2),
                ShiftScaleRotate(shift_limit=0.03,rotate_limit=15,scale_limit=0.02,p=1),Cutout(p=0.5)])
transform = create_transform(aug)    

gen_train = DataGenerator(Ids_train,newWhale_train,transform,color=color,HalfBatch=8)
gen_val = DataGenerator(Ids_val,newWhale_val,transform,color=color,HalfBatch=8)

### build model

In [5]:
# conv_base = DenseNet121(weights='imagenet',include_top=False,pooling='max')
#conv_base = Xception_greyscale((256,256,1),'max',False)
conv_base = Xception_reduced((256,256,1))

In [6]:
train_model,feature_model = create_model(1e-3,l2_distance,margin_loss_fun_factory(0.1,5),\
                                         conv_base,color,[512,512],['relu',None])

### training

In [7]:
# conv_base = Xception_reduced((256,256,1))
# train_model,feature_model = create_model(1e-3,l2_distance,margin_loss_fun_factory(0.1,5),\
#                                         conv_base,color,[512,512],['relu',None])
start = time.time()
history = train_model.fit_generator(
          gen_train,
          validation_data = gen_val,
          epochs=5,
          use_multiprocessing=True,workers=2,max_queue_size=20)
end = time.time()
print('time:{}'.format(end - start))

Epoch 1/5
714/714 [==============================] - 248s 348ms/step - loss: -0.3484 - val_loss: -0.3115
Epoch 2/5
714/714 [==============================] - 263s 369ms/step - loss: -0.5330 - val_loss: -0.8432
Epoch 3/5
714/714 [==============================] - 269s 376ms/step - loss: -0.6259 - val_loss: -0.7189
Epoch 4/5
714/714 [==============================] - 269s 377ms/step - loss: -0.7074 - val_loss: -0.9012
Epoch 5/5
714/714 [==============================] - 362s 508ms/step - loss: -0.7410 - val_loss: -0.8922
time:1413.8876395225525


In [ ]:
start = time.time()
history = train_model.fit_generator(
          gen_train,
          validation_data = gen_val,
          epochs=5,
          use_multiprocessing=True,workers=2,max_queue_size=20)
end = time.time()
print('time:{}'.format(end - start))

Epoch 1/5
271/714 [==========>...................] - ETA: 3:37 - loss: -0.7762

In [17]:
# create_model(5e-4,dot_distance,exp_loss,Xception_greyscale((256,256,1),'max',False),color)
start = time.time()
history = train_model.fit_generator(
          gen_train,
          validation_data = gen_val,
          epochs=5,
          use_multiprocessing=True,workers=2,max_queue_size=20)
end = time.time()
print('time:{}'.format(end - start))

Epoch 1/5
714/714 [==============================] - 165s 232ms/step - loss: 0.9346 - val_loss: 3.0979
Epoch 2/5
714/714 [==============================] - 165s 231ms/step - loss: 0.9273 - val_loss: 2.7641
Epoch 3/5
714/714 [==============================] - 165s 231ms/step - loss: 0.9310 - val_loss: 2.9462
Epoch 4/5
714/714 [==============================] - 165s 231ms/step - loss: 0.9089 - val_loss: 5.3465
Epoch 5/5
714/714 [==============================] - 165s 231ms/step - loss: 0.9125 - val_loss: 4.3415
time:824.5533976554871


In [39]:
# create_model(5e-2,dot_sigmoid_distance,exp_loss,Xception_greyscale((256,256,1),'max',False),color)
# this just push everything to huge negative number. So after sigmoid all zero

In [5]:
# create_model(5e-4,sigmoid_dot_distance,exp_loss,Xception_greyscale((256,256,1),'max',False),color)
# loss does not decrease

In [12]:
# create_model(1e-3,dot_distance_neg,cross_entropy_loss,Xception_greyscale((256,256,1),'max',False),color)
start = time.time()
history = train_model.fit_generator(
          gen_train,
          validation_data = gen_val,
          epochs=5,
          use_multiprocessing=True,workers=2,max_queue_size=20)
end = time.time()
print('time:{}'.format(end - start))

Epoch 1/5
714/714 [==============================] - 90s 126ms/step - loss: 0.6287 - val_loss: 2.8820
Epoch 2/5
714/714 [==============================] - 134s 188ms/step - loss: 0.6232 - val_loss: 1.9558
Epoch 3/5
714/714 [==============================] - 166s 232ms/step - loss: 0.6251 - val_loss: 2.0248
Epoch 4/5
714/714 [==============================] - 166s 232ms/step - loss: 0.6239 - val_loss: 2.5173
Epoch 5/5
714/714 [==============================] - 166s 232ms/step - loss: 0.6304 - val_loss: 2.0597
time:721.7185816764832


In [8]:
# train_model,feature_model = create_model(1e-3,l2_distance,margin_loss_fun_factory(0.1,5),\
#                                         conv_base,color)
start = time.time()
history = train_model.fit_generator(
          gen_train,
          validation_data = gen_val,
          epochs=5,
          use_multiprocessing=True,workers=2,max_queue_size=20)
end = time.time()
print('time:{}'.format(end - start))

Epoch 1/5
714/714 [==============================] - 89s 125ms/step - loss: -0.5730 - val_loss: -0.6638
Epoch 2/5
714/714 [==============================] - 85s 119ms/step - loss: -0.6436 - val_loss: -0.6496
Epoch 3/5
714/714 [==============================] - 132s 184ms/step - loss: -0.6517 - val_loss: -0.6050
Epoch 4/5
714/714 [==============================] - 164s 230ms/step - loss: -0.6505 - val_loss: -0.6908
Epoch 5/5
714/714 [==============================] - 164s 230ms/step - loss: -0.6815 - val_loss: -0.6497
time:634.2249140739441


Fine-Tune

In [12]:
feature_model.layers[0].trainable=True

In [14]:
for layer in feature_model.layers[0].layers:
    if layer.name in ['block14_sepconv2','block14_sepconv1']:
        layer.trainable = True
    else:
        layer.trainable = False

In [19]:
train_model.compile(loss=margin_loss_fun_factory(0.1,5),optimizer=tf.keras.optimizers.Adam(lr=1e-4))

In [20]:
start = time.time()
history = train_model.fit_generator(
          gen_train,
          validation_data = gen_val,
          epochs=5,
          use_multiprocessing=True,workers=2,max_queue_size=20)
end = time.time()
print('time:{}'.format(end - start))

Epoch 1/5
714/714 [==============================] - 180s 252ms/step - loss: -0.8127 - val_loss: -0.7030
Epoch 2/5
714/714 [==============================] - 178s 250ms/step - loss: -0.9215 - val_loss: -0.7280
Epoch 3/5
714/714 [==============================] - 179s 250ms/step - loss: -1.0445 - val_loss: -0.6875
Epoch 4/5
714/714 [==============================] - 179s 251ms/step - loss: -1.0952 - val_loss: -0.5705
Epoch 5/5
714/714 [==============================] - 179s 250ms/step - loss: -1.0889 - val_loss: -0.6476
time:895.5712113380432


In [21]:
train_model.save('Models/train_l2_distance_margin_loss_fun_factory(0.1,5).h5')

In [22]:
feature_model.save('Models/feature_l2_distance_margin_loss_fun_factory(0.1,5).h5')